In [9]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"

In [10]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [11]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)


In [12]:
dataset = generate_dataset()

with open("dataset.json", "w") as f:
        json.dump(dataset, f, indent=2)

In [19]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [21]:
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)
    

In [22]:
def run_test_case(test_case):
    """Calls run_prompt, then grades thee result"""
    output = run_prompt(test_case)

    # TODO - Grading
    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoing": reasoning
    }

In [27]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)
    
    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [28]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 6.666666666666667


In [29]:
print(json.dumps(results, indent=2))

[
  {
    "output": "Here's a regex pattern to validate AWS IAM username format:\n\n```regex\n^[a-z][a-z0-9]{0,63}$\n```\n\nLet's break down the regex pattern:\n\n- `^` - Start of the string\n- `[a-z]` - First character must be a lowercase letter\n- `[a-z0-9]{0,63}` - Followed by 0-63 characters that can be lowercase letters or numbers\n- `$` - End of the string\n\nThis regex ensures:\n- Starts with a lowercase letter\n- Contains only lowercase letters and numbers\n- Maximum length of 64 characters\n\nExample validation in Python:\n\n```python\nimport re\n\ndef validate_iam_username(username):\n    pattern = r'^[a-z][a-z0-9]{0,63}$'\n    return bool(re.match(pattern, username))\n\n# Test cases\nprint(validate_iam_username('john-doe'))     # True\nprint(validate_iam_username('johndoe123'))   # True\nprint(validate_iam_username('John-Doe'))     # False (uppercase)\nprint(validate_iam_username('1johndoe'))     # False (starts with number)\nprint(validate_iam_username('a' * 65))       # Fa